#### Section 4: In this section, we will use the textual concept maps to generate:

    the two updated profiles of Max and Samanta
    an outline that summarizes and lists the main concepts from the concept maps in an organized manner



In [16]:
#generiamo due profili partendo dalle due mappe concettuali testuali 
import gradio as gr
import ollama
import os
import json
from pathlib import Path

modello_max='gemma3:12b'
modello_sam='gemma3:12b'
modello_pol='gemma3:12b'

css = """
button:active {
    background-color: #ff6600 !important;  /* colore di sfondo quando premuto */
    color: white !important;               /* colore del testo quando premuto */
    transform: scale(0.97);                /* leggero effetto di "pressione" */
}
"""

BASE_DIR = "progetti"
def profilo_discussione_max(map1, domanda, lingua):
    global messages
    
    istruzioni = (
        f"Sei un esperto nella creazione di profili per dibattiti, partendo da una mappa concettuale."
        f" Ti verrà fornita una mappa concettuale che descrive concetti, relazioni e categorie tematiche."
        f" Il tuo compito è trasformarla in un profilo strutturato dell'attore, basandoti sui concetti individuati.\n\n"

        f"### Regole per costruire il profilo:\n"
        f"1. **Identifica le aree di competenza**: Raggruppa i concetti principali e le materie in cui l'attore è preparato.\n"
        f"2. **Unifica concetti simili**: Se due concetti sono strettamente correlati (es. 'Meccanica Quantistica' e 'Fisica Quantistica'), trattali come un'unica area.\n"
        f"3. **Distingui tra conoscenze centrali e periferiche**: I concetti principali devono essere evidenziati, mentre quelli secondari possono essere menzionati in sotto-categorie.\n"
        f"4. **Recupera concetti mancanti**: Se nella mappa concettuale mancano elementi utili al dibattito, suggerisci competenze aggiuntive rilevanti.\n\n"

        f"### Struttura del profilo da generare:\n"
        f"- **Nome dell'attore**\n"
        f"- **Aree di competenza**\n"
        f"- **Competenze aggiuntive**\n"
        f"- **Ruolo nel dibattito**\n"
        f"- **Approccio nel dibattito**\n\n"
        f"Rispondi esclusivamente in lingua: {lingua}.\n\n"
        f"### Mappa concettuale:\n{map1}"
    )

    messages = [
        {"role": "user", "content": istruzioni}
    ]
    
    return messages
    
def scaletta_discussione_max(map1, domanda, lingua):
   
    global messages
    messages = [
        {
            'role': 'system',
            'content': (
                f"Sei un esperto nella creazione di scalette per dibattiti basate su mappe concettuali. "
                f"Ti verrà fornita una mappa concettuale che include concetti, relazioni e categorie tematiche. "
                f"Il tuo compito è estrarre una scaletta ricca di spunti,  dettagliata e strutturata per un dibattito o una lezione, seguendo le regole seguenti:\n\n"
                f"### Regole per costruire la scaletta:\n"
                f"1. Inizia con un solo punto introduttivo che presenta il tema: '{domanda}'.\n"
                f"2. Ogni punto deve essere un titolo sintetico.\n"
                f"3. I punti devono essere numerati in ordine logico e progressivo.\n"
                f"4. Non aggiungere commenti, spiegazioni o testo introduttivo.\n"
                f"5. L’ultimo punto deve essere 'Conclusioni'.\n"
                f"6. Rispondi esclusivamente nella lingua: '{lingua}'.\n"
               
            )
        },
        {
            'role': 'user',
            'content': f"Ecco la mappa concettuale:\n{map1}"
        }
    ]

    return messages


def scaletta_discussione_sam(map2, domanda,lingua):
    global messages
    messages = [
    {
        'role': 'system',
        'content': (
            "Sei un esperto nella creazione di scalette per dibattiti basate su mappe concettuali.'{comp2}' "
            "Ti verrà fornita una mappa concettuale che include concetti, relazioni e categorie tematiche. "
            "Il tuo compito è estrarre una scaletta strutturata per un dibattito o una lezione, seguendo le regole seguenti:\n\n"
            
            "### Regole per costruire la scaletta:\n"
            "1. Inizia con un'introduzione che contestualizzi il tema del dibattito contenuto nella domanda: '{domanda}'.\n"
            "2. Dividi la scaletta in punti, mantenendo un ordine logico e didattico.\n"
            "3. Ordina i punti della scaletta in modo logico e progressivo, seguendo un flusso coerente.\n "
            "4. Ogni punto deve essere formulato come un'affermazione precisa e concisa.\n"
            "5. Genera solo una lista numerata di punti che esplorano il tema della domanda, con focus su concetti scientifici chiari e definiti.\n"
            "6. L'ultimo punto della scaletta sono le conclusioni.\n"
            "7. Genera solo la scaletta suddivisa in punti e nessun commento.\n"
            " Attenzione la tua risposta deve essere data nella lingua '{lingua}'"
        )
    },
    {
        'role': 'user',
        'content': f"Ecco la mappa concettuale:\n{map2}"
    }
]

    return messages



def profilo_discussione_sam(map2, domanda,lingua):
    global messages
    messages = [
    {
        'role': 'system',
        'content': (
            "Sei un esperto nella creazione di profili per dibattiti, partendo da una mappa concettuale."
            " Ti verrà fornita una mappa concettuale che descrive concetti, relazioni e categorie tematiche."
            " Il tuo compito è trasformarla in un profilo strutturato dell'attore, basandoti sui concetti individuati.\n\n"

            "### Regole per costruire il profilo:\n"
            "1. **Identifica le aree di competenza**: Raggruppa i concetti principali e le materie in cui l'attore è preparato.\n"
            "2. **Unifica concetti simili**: Se due concetti sono strettamente correlati (es. 'Meccanica Quantistica' e 'Fisica Quantistica'), trattali come un'unica area.\n"
            "3. **Distingui tra conoscenze centrali e periferiche**: I concetti principali devono essere evidenziati, mentre quelli secondari possono essere menzionati in sotto-categorie.\n"
            "4. **Recupera concetti mancanti**: Se nella mappa concettuale mancano elementi utili al dibattito, suggerisci competenze aggiuntive rilevanti.\n\n"

            "### Struttura del profilo da generare:\n"
            "- **Nome dell'attore**\n"
            "- **Aree di competenza**: Un elenco organizzato dei concetti e discipline principali.\n"
            "- **Competenze aggiuntive**: Concetti non presenti nella mappa, ma utili per arricchire il dibattito.\n"
            "- **Ruolo nel dibattito**: Descrivi come l'attore utilizzerà le sue competenze nel confronto con altri partecipanti.\n"
            "- **Approccio nel dibattito**: Indica il modo in cui l'attore argomenta (es. rigoroso e matematico, interdisciplinare, provocatorio, etc.)."
            " Attenzione la tua risposta deve essere data nella lingua '{lingua}'"
        )
    },
    {
        'role': 'user',
        'content': f"Ecco la mappa concettuale:\n{map2}"
    }
]

    return messages
    
# Carica il percorso salvato
CONFIG_PATH = Path(os.getenv("APPDATA")) / "MaGo" / "config.json"

def load_project_path():
    """Carica il percorso dal file di configurazione"""
    if not CONFIG_PATH.exists():
        return None
    
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
    return config.get("project_path")

# Verifica all'avvio
project_path = load_project_path()
if not project_path or not Path(project_path).exists():
    raise RuntimeError(
        "🛑 Percorso non configurato o non valido!\n"
        "Esegui prima il notebook 'installer.ipynb' e imposta la cartella corretta."
    )

# Imposta la directory di lavoro
os.chdir(project_path)
print(f"✔️ Directory di lavoro impostata a: {project_path}")


global project_path






def genera_profilo_max():
    global messages,esp1,map1,domanda,progetto,lingua,project_path
    
    chat_stato_max = []
    
    messages=profilo_discussione_max(map1, domanda,lingua)
    messages.append({"role": "user", "content": f"Rispondi sempre in {lingua}."})
    response = ollama.chat(model=modello_max, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    print(f" Max: {response['message']['content']}\n")
    chat_stato_max.append(("Max", response['message']['content']))

    prof=response['message']['content']
    prof=esp1+prof
    
         # Percorso dove salvare la scaletta
    project_dir = Path(project_path) / "progetti" / progetto
    
    # Scrivi la risposta di Max su un file di testo
    with open(project_dir / 'prof1.txt', 'w', encoding="utf-8") as file:
        file.write(prof)  # Scrive la risposta nel file


    return prof

def genera_scaletta_max():
    global map1,domanda,messages,progetto,lingua,project_path
    messages=[]
    chat_stato_max = []
    #profilo=comp1
    messages=scaletta_discussione_max(map1, domanda,lingua)

    messages.append({"role": "user", "content": f"Rispondi sempre in {lingua}."}) 
    response = ollama.chat(model=modello_max, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Max: {response['message']['content']}\n")
    chat_stato_max.append(("Max", response['message']['content']))

    scal=response['message']['content']

            # Percorso dove salvare la scaletta
    project_dir = Path(project_path) / "progetti" / progetto
    
    # Scrivi la risposta di Max su un file di testo
    with open(project_dir / 'scal1.txt', 'w', encoding="utf-8") as file:
        file.write(scal)  # Scrive la risposta nel file

    return scal

def genera_profilo_sam():
    global messages,esp2,map2,domanda,progetto,lingua,project_path
    messages=[]
    chat_stato_sam = []
 
    messages=profilo_discussione_sam(map2, domanda,lingua)

    messages.append({"role": "user", "content": f"Rispondi sempre in {lingua}."})
    response = ollama.chat(model=modello_sam, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Samanta: {response['message']['content']}\n")
    chat_stato_sam.append(("Samanta", response['message']['content']))

    prof=response['message']['content']
    prof=esp2+prof
    
         # Percorso dove salvare la scaletta
    project_dir = Path(project_path) / "progetti" / progetto
    
    # Scrivi la risposta di Max su un file di testo
    with open(project_dir / 'prof2.txt', 'w', encoding="utf-8") as file:
        file.write(prof)  # Scrive la risposta nel file


    return prof

def genera_scaletta_sam():
    global map2,domanda,messages,progetto,lingua,project_path
    messages=[]
    chat_stato_max = []
    #profilo=comp1
    messages=scaletta_discussione_sam(map2, domanda,lingua)

    messages.append({"role": "user", "content": f"Rispondi sempre in {lingua}."})
    response = ollama.chat(model=modello_sam, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Samanta: {response['message']['content']}\n")
    chat_stato_max.append(("Samanta", response['message']['content']))

    scal=response['message']['content']

     # Percorso dove salvare la scaletta
    project_dir = Path(project_path) / "progetti" / progetto


    # Salva la scaletta nel file
    with open(project_dir / 'scal2.txt', 'w', encoding="utf-8") as file:
        file.write(scal)


    return scal  


def carica_progetto(nome_progetto):
    global esp1,esp2,map1,map2,domanda,progetto,scal1,scal2,prof1,prof2,lingua,project_path
    progetto=nome_progetto

    project_dir = Path(project_path) / "progetti" / nome_progetto

    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", ""
    
    def leggi_file(nome_file):
        path = os.path.join(project_dir, nome_file)
        try:
            with open(path, 'r', encoding='utf-8') as f:
                contenuto = f.read().strip()
                return contenuto if contenuto else ""  # Restituisce stringa vuota se il file è vuoto
        except FileNotFoundError:
            print(f"Attenzione: il file '{nome_file}' non è stato trovato.")
            return ""
        except Exception as e:
            print(f"Errore nella lettura del file '{nome_file}': {e}")
            return ""

    # Carica le variabili, gestendo gli errori in modo semplice
    esp1 = leggi_file("esp1.txt")
    esp2 = leggi_file("esp2.txt")
    map1 = leggi_file("map1.txt")
    map2 = leggi_file("map2.txt")
    scal1 = leggi_file("scal1.txt")
    scal2 = leggi_file("scal2.txt")
    prof1 = leggi_file("prof1.txt")
    prof2 = leggi_file("prof2.txt")
    lingua = leggi_file("lingua.txt")



    
    domanda = leggi_file("domanda.txt")

    return esp1,esp2,map1, map2, domanda,prof1,scal1,prof2,scal2


def aggiorna_progetti():
    """Restituisce la lista delle cartelle dei progetti."""
    global project_path
    project_dir = Path(project_path) / "progetti" 
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)  # Crea la directory se non esiste
    return [d for d in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, d))]

    

def avvia_interfaccia():
     # Aggiungi questo componente
    html_output = gr.HTML(label="Mappa Concettuale")
   
    
    
    with gr.Blocks(css=css) as demo:
        gr.Markdown("# Update Max and Samanta's profiles and generate a simplified version of the maps ")
       
        with gr.Row():
            Progetto = gr.Dropdown(label="Select a project", choices=aggiorna_progetti())
            btn_carica = gr.Button("Load Project")

        domanda_output = gr.Textbox(label="QUESTION", interactive=False)
        with gr.Row():
            esp1_output = gr.Textbox(label="Initial profile of Max", interactive=False)
            esp2_output = gr.Textbox(label="Initial profile of samanta", interactive=False)
        with gr.Row():
            map1_output = gr.Textbox(label="Concept map of Max", interactive=False)
            map2_output = gr.Textbox(label="Concept map of Samanta", interactive=False)
     
      
        with gr.Row():  # Bottoni per generare le mappe grafiche
            btn3 = gr.Button("Generate Max's Profile")
            btn4 = gr.Button("Generate Samanta's Profile")

            
        with gr.Row():    
            profilo_output_max= gr.Textbox(label="Max's Profile", interactive=False)
            profilo_output_sam = gr.Textbox(label="Samanta's Profile", interactive=False)

        
        btn3.click(fn=genera_profilo_max, inputs=[], outputs=[profilo_output_max]
        )
        btn4.click(fn=genera_profilo_sam, inputs=[], outputs=[profilo_output_sam]
       )
        with gr.Row():  # Bottoni per generare le scalette
            btn5 = gr.Button("Generate Max's Outline")
            btn6 = gr.Button("Generate Samanta's Outline")

        with gr.Row():    
            scaletta_output_max= gr.Textbox(label="Max's Outline", interactive=False)
            scaletta_output_sam = gr.Textbox(label="Samanta's Outline", interactive=False)

        btn_carica.click(fn=carica_progetto, inputs=[Progetto], 
        outputs=[esp1_output,esp2_output,map1_output, map2_output, domanda_output,profilo_output_max,scaletta_output_max,profilo_output_sam,scaletta_output_sam])
      
        btn5.click(fn=genera_scaletta_max, inputs=[], outputs=[scaletta_output_max]
        )
        btn6.click(fn=genera_scaletta_sam, inputs=[], outputs=[scaletta_output_sam]
        )
   
    
    demo.launch()

if __name__ == "__main__":
    avvia_interfaccia()

✔️ Directory di lavoro impostata a: C:\Users\maian\notebook\py10\MaGo
* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
